In [33]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pathlib import Path
import time

filename = "NammaMetro_Ridership_Dataset.csv"

In [34]:
# Selenium options required to create a 'headless' browser
options = Options()
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.37")

driver = webdriver.Chrome(options=options)

In [35]:
# Load ridership page from BMRCL website
driver.get("https://english.bmrc.co.in/ridership/")

# Click on Kannada toggle button to load English results
time.sleep(10)  # Allow extra time for JavaScript to load translated data
result = driver.find_element(By.CLASS_NAME, "link.top-navcustom-text").click()

In [36]:
# Results are published with a lag of about one day. 
# So get the date on the page rather than date.today()
record_date = driver.find_element(By.TAG_NAME, "h3").text
record_date

'Passenger Flow as on 04-04-2025'

In [37]:
# Initialize dict to store ridership data
day_record = {}
day_record['Record Date'] = [record_date.split()[-1]]  # Extracting date part
day_record

{'Record Date': ['04-04-2025']}

In [38]:
# Parse html for remaining data points and store in pandas dataframe
data_points = driver.find_elements(By.CLASS_NAME, "features-card.achivement-area.bg-color")

for l1 in data_points:
    for l2 in l1.text.split('\n'):
        data = l2.split(': ')
        day_record[data[0]] = [int(data[1])]

driver.quit()

day_record = pd.DataFrame(day_record)
day_record.rename(columns={'Tokens':'Total Tokens'}, inplace=True)
day_record

,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Total Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
0,04-04-2025,419016,418537,136,36,307,202926,10892,249,169503,47705,91362,30436


In [43]:
# Store data in csv file - create file if necessary
filePath = Path(filename)
if filePath.exists() and filePath.is_file():
    day_record.to_csv(filename, mode='a', header=False, index=False, lineterminator='\n')
    print('Appended '+filename)
else:
    day_record.to_csv(filename, mode='w', header=True, index=False, lineterminator='\n')
    print('Created '+filename)

Appended NammaMetro_Ridership_Dataset.csv


In [44]:
# Optimize dataset by removing duplicates and rewrite to file
df = pd.read_csv(filename).drop_duplicates(subset=['Record Date'], keep='last', ignore_index=True)
df[df.loc[:, 'Total Smart Cards':].columns] = df[df.loc[:, 'Total Smart Cards':].columns].astype('Int64')
df.to_csv(filename, mode='w', header=True, index=False, lineterminator='\n')
df.tail()

,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Total Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
127,26-03-2025,449665,448857,207,40,561,191193,20478,76,155934,42766,83068,30100
128,29-03-2025,270949,270298,472,55,124,209774,10861,288,186852,52338,101606,32908
129,30-03-2025,96209,95682,428,30,69,172307,4444,72,129763,35130,69634,24999
130,31-03-2025,208196,207708,228,29,231,208738,9122,0,145793,40258,79349,26186
131,04-04-2025,419016,418537,136,36,307,202926,10892,249,169503,47705,91362,30436
